# テンソルネットワークのべんきょう

西野友年「テンソルネットワーク入門」講談社，2023に出てくる計算を実際にプログラムしてみます．

- 言語はJuliaで


## 第4章 角転送行列

---

### 4脚テンソルの値

詳細については `sec4.角転送行列.pdf` を参照のこと

本書では4脚テンソルの値は次のように決められている．

$$
W_{0010} = W_{1000} = W_{0001} = W_{0100} = 1,
$$
それ以外の配置の場合には
$$
    W_{abcd} = 0
$$
ただし，テキスト中ではテンソルの脚は十字型に配置されているが，この文書では表現できないため，一番左の脚から時計回りに $W_{abcd}$としている．

In [1]:
# 4階のテンソルを作成
W = zeros(Int64, 2, 2, 2, 2)

W[2, 1, 1, 1] = 1
W[1, 2, 1, 1] = 1
W[1, 1, 2, 1] = 1
W[1, 1, 1, 2] = 1

println(W)
println(size(W))

[0 1; 1 0;;; 1 0; 0 0;;;; 1 0; 0 0;;; 0 0; 0 0]
(2, 2, 2, 2)


### p.52 宿題
> $P^{(n)}$ と $C^{(n)}$ を $n=1$ から順番に求めて，縮約 $c^{(n)}=\mathrm{Tr}\left[ C^{(n)} \right]^4$ の値を得るプログラムを組みなさい．\
> $P^{(n+1)}$ や $C^{(n+1)}$ を求めたら，その時点で $P^{(n)}$ と $C^{(n)}$ は不要になるので，捨てても良いことに注意．

まず，部分和，$C^{(1)}_{(a, b)}$ と $P^{(1)}_{(a, b, c)}$ を定めておく．

In [2]:
# 2階のテンソルを作成
C1 = zeros(Int64, 2, 2)
C1[2, 1] = 1
C1[1, 2] = 1

# 3階のテンソルを作成
P1 = zeros(Int64, 2, 2, 2)
P1[2, 1, 1] = 1
P1[1, 2, 1] = 1
P1[1, 1, 2] = 1

1

In [78]:
C1

2×2 Matrix{Int64}:
 0  1
 1  0

In [79]:
P1

2×2×2 Array{Int64, 3}:
[:, :, 1] =
 0  1
 1  0

[:, :, 2] =
 1  0
 0  0

## 計算の概要

1. $C^{(n+1)}$ を $C^{(n)}$ と $P^{(n)}$ から求める処理の実装
   1. First, compute $Q^{(n)}$ from $C^{(n)}$ and $P^{(n)}$
   2. Second, compute $P^{(n+1)}$ from $P^{(n)}$ and $W$
   3. Third, compute $C^{(n+1)}$ from $Q^{(n)}$ and $P^{(n+1)}$
2. $n=1$ から $n$ まで順に $C^{(n)}$ を計算する処理の実装
3. $c^{(n)}=\mathrm{Tr}\left[ C^{(n)} \right]^4$ を計算する

#### 1-1. $C^{(n)}$ を $P^{(n)}$ で部分和を計算する処理

この部分和を $Q^{(n)}$ とおく

In [70]:
"""
Contract P^{(n)} and C^{(n)}

In: 
    P^{(n)}_{μ α i}
        μ = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        α = n*k + (n-1)*k' + ... k'''で，μと同様
        i は (0,1)をとる脚
    C^{(n)}_{α β}: 角転送行列
        α = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        β = n*k + (n-1)*k' + ... k'''で，αと同様
Out:
    Q^{(n)}_{μ γ}
        μ = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        γ = n+1*k + n*k' + ... k'''で, (0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            β と i をまとめて一つの文字にしている
"""
function compute_Q_n(C_n, P_n)
    # μ の 数は 2^n 通り, γ の数は 2^(n+1) 通りである．
    two_to_the_n = size(P_n, 1) # = 2^n
    n = Int(log2(two_to_the_n))
    # initialize CPn
    Q_n = zeros(Int64, 2^n, 2^(n+1))
    # println(size(Q_n))
    # compute CPn
    # μ については，脚をまとめないので，そのまま
    for μ in 1:2^n
        # γ については，脚をまとめるので， P_n+1 と同じように計算する
        for γ in 1:2^(n+1)
            value = 0
            # 生の脚 i について，場合わけをして決める
            if γ <= 2^n
                i = 1
                β = γ
            else
                i = 2
                β = γ - 2^n
            end
            # sum over α
            for α in 1:2^n
                value += C_n[α, β] * P_n[μ, α, i]
            end
            # print("μ = ", μ, ", γ = ", γ, ", i = ", i, ", β = ", β, ", value = ", value, "\n")
            Q_n[μ, γ] = value
        end
    end
    return Q_n
end

# Q1 = compute_Q_n(C1, P1)

compute_Q_n

#### 1-2. $P^{(n+1)}$ を $P^{(n)}$ から計算する処理

In [69]:
"""
Compute4 P^{(n+1)} from P^{(n)}

In: 
    P^{(n)}_{μ ν i}
        μ = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        ν = n*k + (n-1)*k' + ... k'''で，μと同様
        i は (0,1)をとる脚
    W_{a j b i} は，a,j,b,iが(0,1)をとる4本の脚を持つテンソル
Out:
    P^{(n+1)}_{ρ σ j}
        μ = (n+1)*j + n*j' + ... j'''で，(0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            0 から ((n+1)^2 - 1) まで，2^n通りの値を取りうる
        ρ = n+1*k + n*k' + ... k'''で，μと同様
        j は (0,1)をとる脚
"""
function contract_P_np1_W(P_n, W::Array{Int64, 4})
    # 取りうる状態の数は， n^2 通りなので， Pの大きさは 2^n * 2^n * 2 になっている
    two_to_the_n = size(P_n, 1) # = 2^n
    n = Int(log2(two_to_the_n))
    # println("n = ", n)
    # initialize P^{(n+1)}
    P_np1 = zeros(Int64, 2^(n+1), 2^(n+1), 2)
    # println(size(P_np1))
    # compute P^{(n+1)}
    for ρ in 1:2^(n+1); for σ in 1:2^(n+1); for j in 1:2
        value = 0
        # set a 
        if ρ <= 2^n
            a = 1
            μ = ρ
        else
            a = 2
            μ = ρ - 2^n
        end
        # set b
        if σ <= 2^n
            b = 1
            ν = σ
        else
            b = 2
            ν = σ - 2^n
        end
        # sum over i
        for i in 1:2
            value += P_n[μ, ν, i] * W[a, j, b, i]
        end
        # print("ρ = ", ρ, ", σ = ", σ, ", j = ", j, ", a = ", a, ", b = ", b, ", value = ", value, "\n")
        P_np1[ρ, σ, j] = value
    end; end; end
    return P_np1
end

# P2 = contract_P_np1_W(P1, W)

contract_P_np1_W

#### 1-3. $Q^{(n)}$ と $P^{(n+1)}$ の縮約をとって $C^{(n+1)}$ を計算する関数

In [68]:
"""
Contract Q^{(n)} and P^{(n+1)}

In: 
    Q^{(n)}_{α ρ}
        α = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        ρ = n+1*k + n*k' + ... k'''で, (0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            β と i をまとめて一つの文字にしている
    P^{(n+1)}_{ρ σ j}
        ρ = (n+1)*j + n*j' + ... j'''で，(0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            0 から ((n+1)^2 - 1) まで，2^n通りの値を取りうる
        σ = n+1*k + n*k' + ... k'''で，μと同様
        j は (0,1)をとる脚
Out:
    P^{(n+1)}_{ϕ ψ}
        ϕ = (n+1)*j + n*j' + ... j'''で，(0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            0 から ((n+1)^2 - 1) まで，2^n通りの値を取りうる
        σ = (n+1)*j + n*j' + ... j'''で，(0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            0 から ((n+1)^2 - 1) まで，2^n通りの値を取りうる

"""
function contract_Q_n_P_np1(Q_n, P_np1)
    # ρ の 数は 2^(n+1) 通り, α の数は 2^n+1 通りである．
    two_to_the_n = size(Q_n, 1) # = 2^n
    n = Int(log2(two_to_the_n))
    # initialize C_np1
    C_np1 = zeros(Int64, 2^(n+1), 2^(n+1))
    # println(size(C_np1))
    # compute C_np1
    # σ については，脚をまとめないので，そのまま
    for σ in 1:2^(n+1)
        # ϕ については，脚をまとめるので， P_n+1 と同じように計算する
        for ϕ in 1:2^(n+1)
            value = 0
            # 生の脚 j について，場合わけをして決める
            if ϕ <= 2^n
                j = 1
                α = ϕ
            else
                j = 2
                α = ϕ - 2^n
            end
            # sum over ρ
            for ρ in 1:2^(n+1)
                value += Q_n[α, ρ] * P_np1[ρ, σ, j]
                # print("ϕ = ", ϕ, ", σ = ", σ, " ρ = ", ρ, ", j = ", j, ", α = ", α, ", Q_n[α, ρ] * P_np1[ρ, σ, j] = ", Q_n[α, ρ] * P_np1[ρ, σ, j], "\n")
            end
            C_np1[ϕ, σ] = value
        end
    end
    return C_np1
end

# C2 = contract_Q_n_P_np1(Q1, P2)

contract_Q_n_P_np1

## 1. 上記の計算をまとめて，$C^{(n+1)}$ を $C^{(n)}$ と $P^{(n)}$ から計算する

In [71]:
"""
Compute C^{(n+1)} from C^{(n)} and P^{(n)}

1. First, compute Q^{(n)} from C^{(n)} and P^{(n)} 
2. Second, compute P^{(n+1)} from P^{(n)} and W
3. Third, compute C^{(n+1)} from Q^{(n)} and P^{(n+1)}

In: 
    P^{(n)}_{μ α i}
        μ = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        α = n*k + (n-1)*k' + ... k'''で，μと同様
        i は (0,1)をとる脚
    C^{(n)}_{α β}: 角転送行列
        α = n*j + (n-1)*j' + ... j'''で，(0,1)をとるn本の脚を1つの脚にまとめたもの.
            0 から (n^2 - 1) まで，2^n通りの値を取りうる
        β = n*k + (n-1)*k' + ... k'''で，αと同様

Out:
    P^{(n+1)}_{ρ σ j}
        μ = (n+1)*j + n*j' + ... j'''で，(0,1)をとるn+1本の脚を1つの脚にまとめたもの.
            0 から ((n+1)^2 - 1) まで，2^n通りの値を取りうる
        ρ = n+1*k + n*k' + ... k'''で，μと同様
        j は (0,1)をとる脚
"""
function compute_C_np1_from_C_n_P_n(C_n, P_n)
    Q_n = compute_Q_n(C_n, P_n)
    P_np1 = contract_P_np1_W(P_n, W)
    C_np1 = contract_Q_n_P_np1(Q_n, P_np1)
    return C_np1, P_np1
end

C2, P2 = compute_C_np1_from_C_n_P_n(C1, P1)

([2 0 0 1; 0 0 1 0; 0 1 0 0; 1 0 0 0], [1 0 0 1; 0 0 1 0; 0 1 0 0; 1 0 0 0;;; 0 1 0 0; 1 0 0 0; 0 0 0 0; 0 0 0 0])

## 2. $n=1$ から $n$ まで順に $C^{(n)}$ を計算する処理の実装

In [80]:
function compute_C_n(n)
    size_C_n = n - 1
    # initialize
    C = C1
    P = P1
    # succesive computation
    for i in 1:size_C_n
        C_np1, P_np1 = compute_C_np1_from_C_n_P_n(C, P)
        C = C_np1
        P = P_np1
    end
    return C
end

compute_C_n (generic function with 1 method)

In [83]:
C2 = compute_C_n(2)

4×4 Matrix{Int64}:
 2  0  0  1
 0  0  1  0
 0  1  0  0
 1  0  0  0

In [85]:
C2 * C2

4×4 Matrix{Int64}:
 5  0  0  2
 0  1  0  0
 0  0  1  0
 2  0  0  1

## 3. $c^{(n)}=\mathrm{Tr}\left[ C^{(n)} \right]^4$ を計算する

In [98]:
using LinearAlgebra

function compute_c_value(n)
    C_n = compute_C_n(n)
    c_value = tr(C_n * C_n * C_n * C_n)
    size_TN = 2 * n
    println("c' of this model by $size_TN * $size_TN Tensor network: $c_value")
    # return c_value
end

compute_c_value (generic function with 1 method)

---

以上で，大きさ $2n*2n$ のテンソルネットワークについて，縮約を計算するプログラムが完成した

#### (2 * 2) の時

In [103]:
@time compute_c_value(1)

c' of this model by 2 * 2 Tensor network: 2
  0.000152 seconds (34 allocations: 1.227 KiB)


(4*4)

In [104]:
@time compute_c_value(2)

c' of this model by 4 * 4 Tensor network: 36
  0.000274 seconds (49 allocations: 63.758 KiB)


In [105]:
@time compute_c_value(3)

c' of this model by 6 * 6 Tensor network: 6728
  0.000200 seconds (52 allocations: 66.852 KiB)


In [106]:
@time compute_c_value(4)

c' of this model by 8 * 8 Tensor network: 12988816
  0.000225 seconds (56 allocations: 78.922 KiB)


In [107]:
@time compute_c_value(5)

c' of this model by 10 * 10 Tensor network: 258584046368
  0.000550 seconds (62 allocations: 125.375 KiB)


In [110]:
@time compute_c_value(6)

c' of this model by 12 * 12 Tensor network: 53060477521960000
  0.002249 seconds (74 allocations: 340.164 KiB)


In [111]:
@time compute_c_value(7)

c' of this model by 14 * 14 Tensor network: 9111319734685071488
  0.015500 seconds (79 allocations: 1.051 MiB)


In [112]:
@time compute_c_value(8)

c' of this model by 16 * 16 Tensor network: -7903237475306598144
  0.130360 seconds (88 allocations: 3.926 MiB)


In [113]:
@time compute_c_value(9)

c' of this model by 18 * 18 Tensor network: -1679105710397709824
  1.069925 seconds (95 allocations: 15.427 MiB)


In [114]:
@time compute_c_value(10)

c' of this model by 20 * 20 Tensor network: 4668897434672882688
 12.959195 seconds (102 allocations: 61.427 MiB, 0.04% gc time)
